# Zadanie 1 – rozwiązanie Taxi metodą przeszukiwania
Zacznijmy od zainstalowania i zaimportowania potrzebnych bibliotek.

In [ ]:
try:
    import gym
except:
    !pip install gym

In [ ]:
data_url = 'https://www.mimuw.edu.pl/~mm319369/priv/d73890416bec03ff3e2b3756af8c941c/task1-data.zip'

def download_zip(url):
    from zipfile import ZipFile
    try:
        from urllib import urlretrieve
    except:
        from urllib.request import urlretrieve
    from tempfile import mktemp

    filename = mktemp('.zip')
    name, hdrs = urlretrieve(data_url, filename)
    thefile = ZipFile(filename)
    thefile.extractall('')
    thefile.close()
    
import os

if not os.path.exists('taxi_custom.py'):
    download_zip(data_url)
    print("Done downloading")
else:
    print("Files already exist")

Files already exist


In [ ]:
import gym
from __future__ import print_function

In [ ]:
from taxi_custom import TaxiBuilder
TB = TaxiBuilder()

## Obiektowość w Pythonie
Poniższe kody wykorzystują obiektowość w Pythonie. Nie różni się wiele od Javy poza tym, że nie trzeba podawać typów. Krótki przewodnik możecie znaleźć w materiałach [Wydziału Fizyki](http://brain.fuw.edu.pl/edu/index.php/TI/Wstęp_do_programowania_obiektowego).

W skrócie, przykładowy kod wygląda tak:
```python
class Kot(object):
    """Dokumentacja klasy."""
    
    dzwiek = 'miau' # Domyślna wartość pola w klasie.
    
    def __init__(self, wiek):
        """Dokumentacja metody.
           __init__ to konstruktor. Pierwszy argument to zawsze self -- obiekt naszej klasy."""
        self.wiek = wiek # Do pól klasy odwołujemy się zawsze przez self!
        
    def daj_glos(self):
        print(self.dzwiek)
        
    def daj_n_glosow(self, n):
        for i in range(n):
            # Wołając metodę klasy nie podajemy argumentu self jako parametru -- jest on domyślny.
            self.daj_glos() 
        
    def lataj(self):
        raise NotImplementedError # Rzucanie wyjątku; standardowy sposób na placeholder do implementacji.
        
# Tworzenie nowego obiektu: nie ma new.
kot = Kot(10)
# Wywoływanie metody.
kot.daj_glos()
kot.daj_n_glosow(5)
# Mamy dostęp do wszystkich pól (są domyślnie publiczne, dopóki ich nazwa nie zaczyna się od _).
# W Pythonie nie używa się getterów/setterów dopóki są trywialne.
print(kot.wiek)

# Definicja podklasy Kotek dziedziczącej po klasie Kot
class Kotek(Kot):
    def __init__(self, wiek, kolor):
        # Sposób wołania konstruktora nadklasy
        super(Kotek, self).__init__(wiek)
        # W Pythonie3 po prostu  super().__init__(wiek)
        self.kolor = kolor
    
    # Nadpisanie metody
    def lataj(self):
        print("Nie umiem latac, ale mam kolor %s!" % self.kolor)
        
# Tworzenie obiektu
kotek = Kotek(10, 'zielony')
kotek.lataj()
        
``` 

# Klasy problemów

`Problem` jest abstrakcyjną klasą wszystkich problemów przeszukiwania. Możesz samodzielnie zdefiniować klasę problemów jako podklasę `Problem`u. Musisz nadpisać metody `actions` i `results`, które opisują, jak wygląda problem. Stany końcowe podajesz albo w konstruktorze w parametrze `goals` albo przez nadpisanie metody `is_goal`. Jeśli akcje mają niejednostkowy koszt, nadpisz metodę `step_cost`.

In [ ]:
class Problem(object):
    """Abstrakcyjna klasa problemów przeszukiwania."""

    def __init__(self, initial=None):
        """Podaj stan początkowy i opcjonalnie stany końcowe.
           Podklasy mogą mieć dodatkowe argumenty. Patrz wyżej, jak wywołać konstruktor nadklasy."""
        self.initial = initial  # Stan początkowy problemu.

    def actions(self, state):
        "Zwraca listę akcji ze stanu `state`."
        raise NotImplementedError # Do napisania w podklasie!

    def result(self, state, action):
        "Stan, który jest rezultatem wykonania akcji `action` na stanie `state`."
        raise NotImplementedError # Do napisania w podklasie!

    def is_goal(self, state):
        "Zwraca `True`, gdy stan jest rozwiązaniem." 
        raise NotImplementedError # Do napisania w podklasie!

    def step_cost(self, state, action, result=None):
        "Koszt wykonania akcji `action` ze stanu `state`. `result` może być pomocniczne, gdy go znamy."
        return 1 # Do nadpisania, gdy koszty nie są jednostkowe!

## Przykładowy problem: robot sprzątający
Poniżej znajduje się przykładowa implementacja klasy `Problem`.

Jest to problem robota sprzątającego, jak na ćwiczeniach.

Dwa pomieszczenia, każde może być zakurzone, bądź nie. Ruchy to "jedź w prawo", "jedź w lewo" i "sprzątaj"; wszystkie o koszczie 1.

In [ ]:
dirt  = '*'
clean = ' '

class TwoLocationVacuumProblem(Problem):
    """Robot sprządający ma świat z dwiema lokacjami i kurzem.
       Stany wyglądają następująco (pozycja, kurz_w_L, kurz_w_P)."""

    def actions(self, state):
        # Zawsze pozwalamay sprzątać.
        if state[0] == 'L':
            return ('P', 'Sprzataj')
        else:
            return ('L', 'Sprzataj')
    
    def is_goal(self, state):
        return state[1] != dirt and state[2] != dirt
 
    def result(self, state, action):
        "Stan, który jest rezultatem wykonania akcji `action` na stanie `state`."
        (loc, dirtL, dirtP) = state
        if   action == 'L':                       return ('L', dirtL, dirtP)
        elif action == 'P':                       return ('P', dirtL, dirtP)
        elif action == 'Sprzataj' and loc == 'L': return (loc, clean, dirtP)
        elif action == 'Sprzataj' and loc == 'P': return (loc, dirtL, clean) 
        else: raise ValueError('Nieznana akcja: ' + action)

In [ ]:
# Sprawdźmy, jak się zachowuje
problem = TwoLocationVacuumProblem(initial=('L',dirt,dirt))

In [ ]:
state = problem.initial
print("Stan początkowy:", state)
print("Akcje:", problem.actions(state), "koniec:", problem.is_goal(state))
# Zobaczmy, jak będą wyglądały rezultaty poszczególnych akcji
for action in problem.actions(state):
    print("Akcja:", action, "->", problem.result(state, action))

Stan początkowy: ('L', '*', '*')
Akcje: ('P', 'Sprzataj') koniec: False
Akcja: P -> ('P', '*', '*')
Akcja: Sprzataj -> ('L', ' ', '*')


### Testy
Komórki poniższego typu pozwolą na automatyczną weryfikację poprawności rozwiązania.
Jeśli nie rzucą wyjątku, oznacza, że (prawdopodobnie) wszystko jest ok.

Proszę ich nie modyfikować. Rozwiązania muszą przechodzić wszystkie.

In [ ]:
p_test = TwoLocationVacuumProblem(initial=('L',dirt, dirt))
assert p_test.actions(p_test.initial) == ('P', 'Sprzataj')
l_state = ('L', clean, clean)
assert p_test.result(l_state, 'L') == l_state
assert p_test.result(l_state, 'P') == ('P', clean, clean)
assert p_test.is_goal(p_test.initial) == False
assert p_test.is_goal(l_state) == True
print("Sukces!")

Sukces!


```







```

## Zadanie A: Problem znalezienia ścieżki na planszy
Oto pierwsza część naszego rozwiązania – określenie problemu. Plan naszego agenta podzielimy na dwie fazy:
1. dojechanie do klienta,
2. dojechanie do miejsca docelowego.

Dlatego nasza implementacja klasy Problem będzie dość ogólna i dotyczyła jedynie znajdywanie trasy na mapie między dwoma punktami. Klasa powinna przyjmować w konstruktorze dodatkowo parametr `board` – planszę z naszego środowiska taksówkowego (`env.env.desc`) i ignorować pozycje punktów R, G, Y i B. Stanami niech będą jedynie współrzędne (`y`, `x`).

**Problem nie może odwoływać się do metod środowiska** – powinien być samodzielny i służyć będzie do planowania rozwiązania.


Zaimplementuj odpowiednie metody. Zapoznaj się wcześniej z użyciem tej klasy poniżej.

In [ ]:
# Przypomnijmy, że pole trzeba sprawdzić porównując z bajtami (prefix b)
env_6 = gym.make('Taxi_custom-six-v0')
state = env_6.reset()
print("Akcje:", env_6.env.actions)
board = env_6.env.desc
print(board)
print("Zajęte dobrze:", board[0][0] == b'x', "zajęte źle:", board[0][0] == 'x')

Akcje: ['South', 'North', 'East', 'West', 'Pickup', 'Dropoff']
[[b'x' b'x' b'x' b'x' b'x' b'x' b'x' b'x']
 [b'x' b'R' b'x' b'x' b'x' b'.' b'G' b'x']
 [b'x' b'.' b'.' b'.' b'.' b'.' b'x' b'x']
 [b'x' b'.' b'.' b'x' b'x' b'.' b'x' b'x']
 [b'x' b'.' b'.' b'.' b'.' b'.' b'x' b'x']
 [b'x' b'.' b'x' b'x' b'x' b'x' b'x' b'x']
 [b'x' b'Y' b'.' b'.' b'.' b'.' b'B' b'x']
 [b'x' b'x' b'x' b'x' b'x' b'x' b'x' b'x']]
Zajęte dobrze: True zajęte źle: False


In [ ]:
class PathFindingProblem(Problem):
    """Klasa do znajdywania ścieżki na mapie."""

    def __init__(self, initial, goal, board):
        """Parametry:
           - initial: wsp. początkowa (y,x),
           - goal: wsp. końcowe (y,x),
           - board: plansza."""
        
        super(PathFindingProblem, self).__init__(initial)
        self.goal = goal
        # YOUR CODE HERE 
        self.board = board

    def actions(self, state):
        """Zwraca listę akcji ze stanu `state`.
           Powinny łatwo tłumaczyć się na akcje w środowisku taksówki."""
        # YOUR CODE HERE
        directions = ['South', 'North', 'East', 'West']
        wyn = list()
        # Miałem zamiar zastąpić elif instrukcją match-case, natomiat nie wiem czy została już wprowadzona do pythona 
        # (tutaj instrukcja ta mi nie działała).
        for direction in directions:
          if direction == 'South':
            p_y = state[0] + 1
            p_x = state[1]
          elif direction == 'North':
            p_y = state[0] - 1
            p_x = state[1]
          elif direction == 'East':
            p_y = state[0]
            p_x = state[1] + 1
          elif direction == 'West': 
            p_y = state[0]
            p_x = state[1] - 1

          if ((direction == 'South') and
             (self.board[p_y][p_x] != b'x')):
            wyn.append(0)
          elif ((direction == 'North') and
               (self.board[p_y][p_x] != b'x')):
            wyn.append(1)
          elif ((direction == 'East') and
               (self.board[p_y][p_x] != b'x')):
            wyn.append(2)
          elif ((direction == 'West') and
               (self.board[p_y][p_x] != b'x')):
            wyn.append(3)
        return(wyn)
        
    def result(self, state, action):
        "Stan, który jest rezultatem wykonania akcji `action` na stanie `state`."
        # YOUR CODE HERE
        (y, x) = state
        if action == 0:
            y += 1
        elif action == 1:
            y -= 1
        elif action == 2:
            x += 1
        elif action == 3:
            x -= 1
        else:
          raise ValueError("Nieznana akcja: " + action)
        return (y, x)

    def is_goal(self, state):
        "Zwraca `True`, gdy stan jest rozwiązaniem." 
        return state == self.goal # Mamy jeden stan docelowy

    def step_cost(self, state, action, result=None):
        "Koszt wykonania akcji `action` ze stanu `state`. `result` może być pomocniczne, gdy go znamy."
        # CODE
        return 1

### W poniższych komórkach możesz potestować działanie tej klasy

In [ ]:
# Zadanie: dojechać z (3,2) do (1,1) [R]
problem = PathFindingProblem((3,2), (1,1), env_6.env.desc)

In [ ]:
state = problem.initial
print("Początkowy:", state)
print("Akcje ze stanu początkowego", problem.actions(state))

Początkowy: (3, 2)
Akcje ze stanu początkowego [0, 1, 3]


In [ ]:
print("Akcje ze stanu (1,1):", problem.actions((1,1)))

Akcje ze stanu (1,1): [0]


In [ ]:
print("Czy dojście do (1,1) jest rozwiązaniem?", problem.is_goal((1,1)))
print("Następny stan po startowym:", problem.result(state, problem.actions(state)[0]))

Czy dojście do (1,1) jest rozwiązaniem? True
Następny stan po startowym: (4, 2)


In [ ]:
# Miejsce na Twoje zabawy

In [ ]:
# Użyj +, by dodać nowe komórki

### Testy jednostkowe PathFindingProblem
(Nie wnikające w szczegóły implementacji i dobór akcji.)

In [ ]:
# Koszt jest jednostkowy itp.
fr = 6, 1
to = 6, 6
_, env_6 = TB.get_instance(6)
p_test = PathFindingProblem(fr, to, env_6.env.desc)

assert isinstance(p_test, Problem)
for a in p_test.actions(fr):
    r = p_test.result(fr, a)
    if r != fr:
        assert p_test.step_cost(fr, a, r) == 1
print("Sukces!")

Sukces!


In [ ]:
# Akcje coś znaczą – między sąsiednimi polami istnieją akcje które między nimi prowadzą
# Oraz nie wchodzimy w ścianę
fr = 5, 1
to = 6, 1
_, env_6 = TB.get_instance(6)
p_test = PathFindingProblem(fr, to, env_6.env.desc)

ok = False
for a in p_test.actions(fr):
    r = p_test.result(fr, a)
    assert r not in ((5,0),(5,2)), "Wchodzimy w ścianę"
    if r == to and p_test.is_goal(r):
        ok = True
assert ok
print("Sukces!")

Sukces!


In [ ]:
# Między 6,6 a 6,1 istnieje sensowna ścieżka
fr = 6, 6
to = 6, 1
_, env_6 = TB.get_instance(6)
p_test = PathFindingProblem(fr, to, env_6.env.desc)

st = p_test.initial
seen = set()
for i in range(10):
    seen.add(st)
    for a in p_test.actions(st):
        r = p_test.result(st, a)
        assert p_test.step_cost(st, a, r) == 1
        assert p_test.is_goal(r) == (r == to)
        if r not in seen:
            st = r
            break
    else:
        raise AssertionError("Bez ruchu")
    if r == to:
        break
else:
    raise AssertionError("Nie znaleziono ścieżki")
print("Sukces!")

Sukces!


In [ ]:
# Niezależność od zmiennych globalnych
_, env_t = TB.get_instance(10)
p_test = PathFindingProblem(fr, to, env_t.env.desc)
assert len(p_test.actions((2,2))) == 4 # Wszystkie akcje z pola 2,2
print("Sukces!")

Sukces!




<br><br><br>

# Zadanie B: Węzły
Węzeł reprezentuje stan w drzewie przeszukiwania. Poza obecnym stanem, wskazuje na stan, z którego do niego przyszliśmy, oraz akcję, jaką tego dokonaliśmy. Pamięta też całkowity koszt dojścia do niego i może jakieś dodatkowe wartości. **Uwaga:** całkowity koszt należy pamiętać w momencie tworzenia węzła i liczenie go nie powinno rekurencyjnie wywoływać żadnej funkcji.

W razie potrzeb, możecie rozszerzyć tę klasę o dodatkowe metody/argumenty.

In [ ]:
class Node(object):
    """Węzeł w drzewie przeszukiwania.
       Jeśli do tego samego stanu można dojść na dwa sposoby, będą to oddzielne węzły."""

    def __init__(self, state, previous=None, action=None, step_cost=1):
        """Tworzy nowy węzeł o stanie `state` na podstawie poprzedniego `previous` po akcji `action`
           i koszcie `step_cost`.
           Węzeł startowy ma `previous` i `action` równe None."""
        # YOUR CODE HERE
        self.state = state
        self.previous = previous
        self.action = action
        self.step_cost = 0 
        if previous != None:
          self.step_cost = step_cost + previous.step_cost

    def __repr__(self): 
        """Metoda do zwracająca tekstową reprezentację węzła.
           Wołana automatycznie, np. gdy zrobimy na nim print()."""
        # YOUR CODE HERE
        if self.previous == None:
          return repr("Obecny stan: " + str(self.state) + 
                      ", obecny koszt dojśćia: " + str(self.step_cost) + 
                      ", stan poprzednika: " + 
                      "poprzedni stan był stanem początkowym")
        else:
          return repr("Obecny stan: " + str(self.state) + 
                      ", obecny koszt dojśćia: " + str(self.step_cost) + 
                      ", stan poprzednika: " + str(self.previous.state))
        
    def __lt__(self, other):
        """Metoda wołana przy porównywaniu dwóch węzłów operatorem <. Zwraca True, gdy self < other.
           Przydatna do rozwiązywania remisów."""
        # YOUR CODE HERE
        return self.step_cost < other.step_cost
        
    def children(self, problem):
        """Rozwija dany węzeł w danym problemie `problem` i zwraca węzły potomne."""
        # Hint: self to po prostu obecny obiekt, więc można podać go jako argument do Node(..., self, ...).
        # YOUR CODE HERE
        wyn = list()
        for action in problem.actions(self.state):
          wyn.append(Node(problem.result(self.state, action), self, action, 1))
        return wyn

### W poniższych komórkach możesz potestować działanie tej klasy

In [ ]:
# Znów, z (3,2) do (1,1)
problem = PathFindingProblem((3,2), (1,1), env_6.env.desc)

In [ ]:
initial_node = Node(problem.initial)
print("Węzeł ze stanem początkowym:", initial_node)

Węzeł ze stanem początkowym: 'Obecny stan: (3, 2), obecny koszt dojśćia: 0, stan poprzednika: poprzedni stan był stanem początkowym'


In [ ]:
print("Węzły rozwinięte z (3,2)")
initial_node.children(problem)

Węzły rozwinięte z (3,2)


['Obecny stan: (4, 2), obecny koszt dojśćia: 1, stan poprzednika: (3, 2)',
 'Obecny stan: (2, 2), obecny koszt dojśćia: 1, stan poprzednika: (3, 2)',
 'Obecny stan: (3, 1), obecny koszt dojśćia: 1, stan poprzednika: (3, 2)']

In [ ]:
# Miejsce na Twoje zabawy

In [ ]:
# Użyj +, by dodać nowe komórki

### Testy jednostkowe Node
(Nie wnikające w szczegóły implementacji i dobór akcji.)

In [ ]:
# Poprawność typu funkcji, rozmiar zwracanych danych
_, env_6 = TB.get_instance(6)
fr = 3, 2
to = 1, 1
p_test = PathFindingProblem(fr, to, env_6.env.desc)
in_test = Node(p_test.initial)
chld_test = in_test.children(p_test)
assert len(chld_test) >= 3, "Za mało rozwiniętych węzłów"
assert (chld_test[0] < chld_test[1]) in (True, False), "Zły typ zwrócony przez __lt__"
print(chld_test)
print("Sukces!")

['Obecny stan: (4, 2), obecny koszt dojśćia: 1, stan poprzednika: (3, 2)', 'Obecny stan: (2, 2), obecny koszt dojśćia: 1, stan poprzednika: (3, 2)', 'Obecny stan: (3, 1), obecny koszt dojśćia: 1, stan poprzednika: (3, 2)']
Sukces!


In [ ]:
# Działanie na problemie Robota
p_test = TwoLocationVacuumProblem(('L', dirt, dirt))
in_test = Node(p_test.initial)
chld_test = in_test.children(p_test)
assert len(chld_test) == 2, "Powinny być dwa węzły rozwinięte z początkowego"
print(chld_test)
print("Sukces!")

["Obecny stan: ('P', '*', '*'), obecny koszt dojśćia: 1, stan poprzednika: ('L', '*', '*')", "Obecny stan: ('L', ' ', '*'), obecny koszt dojśćia: 1, stan poprzednika: ('L', '*', '*')"]
Sukces!


<br><br><br>

# Kolejki
Kolejka jest kolekcją `Węzłów`, która zwraca je przyjmuje oraz zwraca w odpowiedniej kolejności. Poniżej znajduje się ogólna klasa kolejki wraz z opisanymi metodami. Pozostałe kolejki też już są zaimplementowane.

Kolejka FIFO zwraca elementy zgodnie z kolejnością wstawiania. Priorytetowa natomiast, zgodnie z priorytetami... Przedstawiono dwie po implementacje: wydajne i naiwne wraz z przykładem użycia.

**Kolejki nie implementują zmian, które dodaliśmy na zajęciach. Modyfikacje zachowania typu pomijanie odwiedzonych stanów proszę implementować w metodzie ``search`` niżej.**

In [ ]:
class AbstractQueue(object):
    """Ogólna klasa kolejek."""
    
    def __init__(self):
        """Inicjalizuje kolejkę."""
        raise NotImplementedError
        
    def add(self, node):
        """Dodaje węzeł `node` do kolejki."""
        raise NotImplementedError
        
    def pop(self):
        """Zwraca następny element z kolejki wedle strategii."""
        raise NotImplementedError
        
    def __len__(self):
        """Zwraca liczbę węzłów w kolejce. Wołana przez użycie len(obiekt)."""
        raise NotImplementedError

In [ ]:
from collections import deque

class FIFOQueue(AbstractQueue):
    def __init__(self):
        self.queue = deque()
        
    def add(self, node):
        self.queue.append(node)
        
    def pop(self):
        return self.queue.popleft()
        
    def __len__(self):
        return len(self.queue)

class SlowFIFOQueue(AbstractQueue):
    def __init__(self):
        self.queue = []
        
    def add(self, node):
        self.queue.append(node)
        
    def pop(self):
        node = self.queue[0]
        self.queue = self.queue[1:]
        return node
        
    def __len__(self):
        return len(self.queue)

In [ ]:
import heapq

class PriorityQueue(AbstractQueue):
    """Kolejka priorytetowa. Priorytet ustalany jest przez funkcję kosztu.
       Złożoność czasowa to O(n log n), gdzie n to liczba elementów wstawianych/wyjmowanych.
    """
    
    def __init__(self, costfn):
        """Inicjalizuje kolejkę.
           `costfn` : None -> int - jest funkcją kosztu, która przyjmuje węzeł i zwraca dla niego priorytet."""
        self.heap   = []
        self.costfn = costfn
    
    def add(self, node):
        """Dodaje węzeł `node` do kolejki."""
        cost = self.costfn(node)
        heapq.heappush(self.heap, (cost, node))
        
    def pop(self):
        """Zwraca element z kolejki o najmniejszej wartości funkcji kosztu."""
        (cost, node) = heapq.heappop(self.heap)
        return node
    
    def __len__(self):
        """Zwraca liczbę węzłów w kolejce. Wołana przez użycie len(obiekt)."""
        return len(self.heap)

class SlowPriorityQueue(AbstractQueue):
    """Kolejka o złożoności czasowej O(n^2), gdzie n to liczba elementów wstawianych/wyjmowanych."""
    
    def __init__(self, costfn):
        self.queue   = []
        self.costfn = costfn
    
    def add(self, node):
        cost = self.costfn(node)
        self.queue.append((cost, node))
        
    def pop(self):
        (cost, node) = min(self.queue)
        self.queue.remove((cost,node))
        return node
    
    def __len__(self):
        return len(self.queue)

### W poniższych komórkach możesz potestować działanie tej klasy

In [ ]:
problem = PathFindingProblem((3,2), (1,1), env_6.env.desc)
# Węzeł ze stanem początkowym
initial_node = Node(problem.initial)

In [ ]:
# Dodajemy do kolejki FIFO rozwinięcia pierwszego węzła
queue = FIFOQueue()
queue.add(initial_node)
print(len(queue))
node = queue.pop()
for n in node.children(problem):
    queue.add(n)
while queue:
    print(queue.pop())

1
'Obecny stan: (4, 2), obecny koszt dojśćia: 1, stan poprzednika: (3, 2)'
'Obecny stan: (2, 2), obecny koszt dojśćia: 1, stan poprzednika: (3, 2)'
'Obecny stan: (3, 1), obecny koszt dojśćia: 1, stan poprzednika: (3, 2)'


In [ ]:
def costfn(node):
    return 5 # Losowy rzut kostką. Napisz coś sensownego

# Dodajemy do kolejki priorytetowej rozwinięcia pierwszego węzła
queue = PriorityQueue(costfn)
queue.add(initial_node)
print(len(queue))
node = queue.pop()
for n in node.children(problem):
    queue.add(n)
while queue:
    print(queue.pop())

1
'Obecny stan: (4, 2), obecny koszt dojśćia: 1, stan poprzednika: (3, 2)'
'Obecny stan: (2, 2), obecny koszt dojśćia: 1, stan poprzednika: (3, 2)'
'Obecny stan: (3, 1), obecny koszt dojśćia: 1, stan poprzednika: (3, 2)'


In [ ]:
# Miejsce na Twoje zabawy

In [ ]:
# Użyj +, by dodać nowe komórki

### Testy jednostkowe Kolejek
(Nie wnikające w szczegóły implementacji i dobór akcji.)

In [ ]:
# Wrzuć do kolejki rozwinięcie węzła początkowego i wyjmij w odpowiedniej kolejności

def test_fifo(q_test_cls):
    _, env_6 = TB.get_instance(6)
    fr = 3, 2
    to = 1, 1
    p_test = PathFindingProblem(fr, to, env_6.env.desc)
    in_test = Node(p_test.initial)
    q_test = q_test_cls()
    q_test.add(in_test)
    assert len(q_test) == 1
    n_test = q_test.pop()
    assert n_test is in_test, "To nie jest ten sam obiekt"
    assert len(q_test) == 0
    chld_test = in_test.children(p_test)

    for n in chld_test:
        q_test.add(n)
    assert len(q_test) >= 3

    for n in chld_test:
        assert n is q_test.pop(), "Zła kolejkość lub to nie jest ten sam obiekt, który wsadziliśmy"

test_fifo(FIFOQueue)
test_fifo(SlowFIFOQueue)
print("Sukces!")

Sukces!


In [ ]:
# Wrzuć do kolejki priorytetowej rozwinięcie węzła początkowego i wyjmij w odrotnej kolejności
def test_priority(q_test_cls):
    _, env_6 = TB.get_instance(6)
    fr = 3, 2
    to = 1, 1
    p_test = PathFindingProblem(fr, to, env_6.env.desc)
    in_test = Node(p_test.initial)

    prio_test = 15
    def cost_test(node):
        nonlocal prio_test
        prio_test -= 1
        return prio_test

    q_test = q_test_cls(cost_test)
    q_test.add(in_test)
    assert len(q_test) == 1
    n_test = q_test.pop()
    assert n_test is in_test, "To nie jest ten sam obiekt"
    assert len(q_test) == 0

    chld_test = in_test.children(p_test)

    for n in chld_test:
        q_test.add(n)
    assert len(q_test) >= 3

    for n in reversed(chld_test):
        assert n is q_test.pop(), "Zła kolejność lub to nie jest ten sam obiekt, który wsadziliśmy"
        
test_priority(PriorityQueue)
test_priority(SlowPriorityQueue)
print("Sukces!")

Sukces!


<br><br><br>

# Zadanie C: Algorytm przeszukiwania

In [ ]:
def search(problem, queue):
    """Algorytm przeszukiwania parametryzowany problemem i obiektem implementującą kolejkę.
       Zwraca węzeł reprezentujący stan będący rozwiązaniem."""
    # Pamiętaj o eliminacji odwiedzonych stanów.
    # Używaj odpowiednich metod problemu i kolejki
    # YOUR CODE HERE
    node = Node(problem.initial)
    queue.add(node)
    global visited # wykorzystujemy w zadaniu E
    global max_len # wykorzystujemy w zadaniu E
    visited = list()
    visited.append(node.state)
    max_len = len(queue)
    while not problem.is_goal(node.state):
      node = queue.pop()
      for n in node.children(problem):
       if n.state not in visited:
         queue.add(n)
         visited.append(n.state)
      if len(queue) > max_len:
        max_len = len(queue)
    return node

Różne algorytmy przeszukiwania to po prostu parametryzacja ogólnego algorytmu przeszukiwania. Poniższa funkcja nie powinna robić wiele poza użyciem już zdefiniowanych poprzednio.

In [ ]:
def BFS(problem):
    """Algorytm przeszukiwania wszerz używa po prostu kolejki FIFO."""
    return search(problem, FIFOQueue())

In [ ]:
def get_solution(node):
    """Dostaje węzeł i zwraca listę akcji do wykonania w środowisku Taxi (0-3)
       aby dojść ze stanu początkowego do stanu w `node`."""
    # YOUR CODE HERE
    lista_akcji = [node.action]
    while node.previous.previous != None:
      node = node.previous
      lista_akcji.insert(0, node.action)
    return lista_akcji

In [ ]:
# Środowisko z mapą rozmiaru 6
_, env = TB.get_instance(6)
problem = PathFindingProblem((4, 4), (6, 6), env.env.desc)

In [ ]:
res = BFS(problem)
print("Rozwiązanie:", res)
print("Ruchy:", get_solution(res))

Rozwiązanie: 'Obecny stan: (6, 6), obecny koszt dojśćia: 10, stan poprzednika: (6, 5)'
Ruchy: [3, 3, 3, 0, 0, 2, 2, 2, 2, 2]


In [ ]:
# Środowisko z mapą rozmiaru 20 i 100
_, env_20 = TB.get_instance(20)
problem_20 = PathFindingProblem((5, 3), (20, 1), env_20.env.desc)
_, env_100 = TB.get_instance(100)
problem_100 = PathFindingProblem((50, 42), (100, 100), env_100.env.desc)

In [ ]:
res = BFS(problem_20)
print("Rozwiązanie:", res)
print("Ruchy:", get_solution(res))

Rozwiązanie: 'Obecny stan: (20, 1), obecny koszt dojśćia: 37, stan poprzednika: (19, 1)'
Ruchy: [2, 0, 2, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 3, 0, 3, 3, 3, 1, 1, 3, 3, 1, 3, 3, 0, 3, 0, 0, 0]


In [ ]:
# Wolne komórki

Poniższa funkcja może się przydać do sprawdzenia, jak zachowuje się nasz algorytm.

In [ ]:
def plot_map(board, path=None, visited=None):
    """
    Funkja pomagająca wyświetlać działanie naszego algorytmu.
    
    `board`, `path` i `visited` to tablice/listy `n`x`n`,
    które mają 1/True w miejscu kolorowania i 0/False w pozostałych.
    `board` koloruje ściany na czarno,
    `path` -- ścieżkę na czerwono,
    `visited` -- odwiedzone stany na niebiesko.
    
    """
    import matplotlib.pyplot as plt
    import numpy as np
    # Ustawia dobre wyświetlanie w Jupyterze/Colaboratory
    get_ipython().magic('matplotlib inline')
    
    size = max(len(board)/10, 5)
    plt.figure(figsize=(size,size))
    board = np.array(board)
    plt.imshow(board, cmap='gray_r', vmin=0, vmax=1)
    if visited is not None:
        visited = np.clip(np.array(visited) + board, 0, 1)
        plt.imshow(visited, cmap='Blues', alpha=0.7, vmin=0, vmax=1)
    if path is not None:
        plt.imshow(path, cmap='Reds', alpha=0.4, vmin=0, vmax=1)
    plt.show()

In [ ]:
# Miejsce na Twoje zabawy

In [ ]:
# Użyj +, by dodać nowe komórki

## Testy jednostkowe przeszukiwania

In [ ]:
_, env_6 = TB.get_instance(6)
fr = (4, 4)
to = (6, 6)
p_test = PathFindingProblem(fr, to, env_6.env.desc)

res_test = BFS(p_test)
assert get_solution(res_test) == [3, 3, 3, 0, 0, 2, 2, 2, 2, 2], "BFS niepoprawny"

# BFS używa funkcji search - usuwamy ze środowiska
old_search_test = search
del search
try:
    BFS(p_test)
    assert False, "BFS nie używa funkcji search"
except NameError:
    pass
finally:
    search = old_search_test

print("Sukces!")

Sukces!


In [ ]:
# Działa też na innych problemach
p_test = TwoLocationVacuumProblem(('L', dirt, dirt))
res_test = BFS(p_test)
assert res_test is not None
print(res_test)
print("Sukces!")

"Obecny stan: ('P', ' ', ' '), obecny koszt dojśćia: 3, stan poprzednika: ('P', ' ', '*')"
Sukces!


In [ ]:
_, env_20 = TB.get_instance(20)
fr = (5, 3)
to = (20, 1)
p_test = PathFindingProblem(fr, to, env_20.env.desc)

res_test = BFS(p_test)
sol_test = get_solution(res_test)
sol_corr = [2, 0, 2, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 3, 0, 3, 3, 3, 1, 1, 3, 3, 1, 3, 3, 0, 3, 0, 0, 0]
assert len(sol_test) == 37, "BFS niepoprawny"
assert sol_test == sol_corr
print("Sukces!")

Sukces!


In [ ]:
_, env_100 = TB.get_instance(100)
fr = (50, 42)
to = (100, 100)
p_test = PathFindingProblem(fr, to, env_100.env.desc)
res_test = BFS(p_test)
assert len(get_solution(res_test)) == 110
print("Sukces!")

Sukces!


## Koniec części pierwszej -- sugeruję wysłać wcześniej

---

## Zadanie D: Przeszukiwanie A* i zachłanne

Zaimplementuj funkcje realizujące przeszukiwanie metodą A* i zachłanną. Skorzystaj z metryki Manhattan (miejskiej). Wykorzystaj zaimplementowane wcześniej kolejki i funkcję search.

In [ ]:
class CostFunction(object):
    """Klasa zawierająca różne heurystyki dla jakiejś kategorii problemów."""
    def __init__(self, problem):
        """Konstruktor tworzy pomocniczy obiekt liczący funkcje kosztu."""
        self.problem = problem
        
    def manhattan(self, state):
        """Zwraca odledległość w metryce Manhattan (miejskiej) ze stanu `state`
           do stanu docelowego w `self.problem` (typu PathFindingProblem)"""
        # YOUR CODE HERE
        x = state
        y = self.problem.goal
        return abs(x[0]-y[0]) + abs(x[1]-y[1])
        
    def __call__(self, node):
        """Metoda wywoływana jest, gdy obiektu używamy jak funkcji tj: `obj(node)`.
           Tę metodę będzie wywoływać kolejka priorytetowa, gdy przekażemy do niej obiekt tego typu.
           `node` - węzeł
        """
        # np. return node.cost
        return node.cost

In [ ]:
class CostFunctionForAstar(CostFunction):
    def __call__(self, node):
        """Implementuje f. kosztu dla węzła w przeszukiwaniu A*."""
        # YOUR CODE HERE
        return self.manhattan(node.state) + node.step_cost

def astar(problem):
    """Wersja algorytmu A* przystosowanego do naszego problemu."""
    # YOUR CODE HERE
    return search(problem, PriorityQueue(CostFunctionForAstar(problem)))

    
class CostFunctionForGreedy(CostFunction):
    def __call__(self, node):
        """Implementuje f. kosztu dla węzła w przeszukiwaniu zachłannym."""
        # YOUR CODE HERE
        return self.manhattan(node.state)
    
# Algorytm zachłanny kieruje się tylko wartością heurystyki
def greedy(problem):
    # YOUR CODE HERE
    return search(problem, PriorityQueue(CostFunctionForGreedy(problem)))

*Przykłady użycia*

In [ ]:
# Środowisko z mapą rozmiaru 6
_, env = TB.get_instance(6)
problem = PathFindingProblem((4, 4), (6, 6), env.env.desc)

In [ ]:
costfn = CostFunctionForAstar(problem)
print("Wartość funkcji kosztu na węźle startowym:", costfn(Node(problem.initial)))

Wartość funkcji kosztu na węźle startowym: 4


In [ ]:
res = astar(problem)
print("Rozwiązanie:", res)
print("Ruchy:", get_solution(res))

Rozwiązanie: 'Obecny stan: (6, 6), obecny koszt dojśćia: 10, stan poprzednika: (6, 5)'
Ruchy: [3, 3, 3, 0, 0, 2, 2, 2, 2, 2]


In [ ]:
res = greedy(problem)
print("Rozwiązanie:", res)
print("Ruchy:", get_solution(res))

Rozwiązanie: 'Obecny stan: (6, 6), obecny koszt dojśćia: 10, stan poprzednika: (6, 5)'
Ruchy: [3, 3, 3, 0, 0, 2, 2, 2, 2, 2]


In [ ]:
# Środowisko z mapą rozmiaru 20 i 100
# 20: optymalne rozwiązanie ma 37 kroków
_, env_20 = TB.get_instance(20)
problem_20 = PathFindingProblem((5, 3), (20, 1), env_20.env.desc)
_, env_100 = TB.get_instance(100)
problem_100 = PathFindingProblem((50, 42), (100, 100), env_100.env.desc)

In [ ]:
res = astar(problem_20)
print("Rozwiązanie:", res)
print("Ruchy:", get_solution(res))

Rozwiązanie: 'Obecny stan: (20, 1), obecny koszt dojśćia: 37, stan poprzednika: (19, 1)'
Ruchy: [2, 0, 2, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 3, 0, 3, 3, 3, 1, 1, 3, 3, 1, 3, 3, 0, 3, 0, 0, 0]


In [ ]:
# Wolne komórki, twórz tutaj

In [ ]:
sol_corr_gr = [3, 0, 0, 0, 2, 0, 2, 2, 0, 0, 2, 0, 2, 2, 0, 0, 0, 0, 2, 2, 0, 0, 3, 0, 3, 3, 3, 1, 1, 3, 3, 1, 3, 3, 0, 3, 0, 0, 0]
res_test = greedy(p_test)
sol_test = get_solution(res_test)

### Testy jednostkowe do D

In [ ]:
_, env_6 = TB.get_instance(6)
fr = (4, 4)
to = (6, 6)
p_test = PathFindingProblem(fr, to, env_6.env.desc)

res_test = astar(p_test)
assert get_solution(res_test) == [3, 3, 3, 0, 0, 2, 2, 2, 2, 2], "Astar niepoprawny"
# Rozwiązanie zachłanne też powinno znaleźć tę samą ścieżkę
res_test = greedy(p_test)
assert get_solution(res_test) == [3, 3, 3, 0, 0, 2, 2, 2, 2, 2], "Greedy niepoprawny"

In [ ]:
_, env_20 = TB.get_instance(20)
fr = (5, 3)
to = (20, 1)
p_test = PathFindingProblem(fr, to, env_20.env.desc)

sol_corr = [2, 0, 2, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 3, 0, 3, 3, 3, 1, 1, 3, 3, 1, 3, 3, 0, 3, 0, 0, 0]
res_test = astar(p_test)
sol_test = get_solution(res_test)
assert len(sol_test) == 37, "AStar niepoprawny"
assert sol_test == sol_corr

sol_corr_gr = [3, 0, 0, 0, 2, 0, 2, 2, 0, 0, 2, 0, 2, 2, 0, 0, 0, 0, 2, 2, 0, 0, 3, 0, 3, 3, 3, 1, 1, 3, 3, 1, 3, 3, 0, 3, 0, 0, 0]
res_test = greedy(p_test)
sol_test = get_solution(res_test)
assert len(sol_test) == 39, "Greedy niepoprawny"
assert sol_test == sol_corr_gr
print("Sukces!")

Sukces!


In [ ]:
_, env_100 = TB.get_instance(100)
fr = (50, 42)
to = (100, 100)
p_test = PathFindingProblem(fr, to, env_100.env.desc)
res_test = astar(p_test)
assert len(get_solution(res_test)) == 110, "AStar niepoprawny"

In [ ]:
_, env_100 = TB.get_instance(100)
fr = (50, 42)
to = (100, 100)
p_test = PathFindingProblem(fr, to, env_100.env.desc)
res_test = greedy(p_test)
assert len(get_solution(res_test)) >= 110, "Greedy niepoprawny"

# Zadanie E: porównanie

Przeanalizuj, ile czasu rzeczywistego zajmuje poszczególnym algorytmom znalezienie trasy, ile stanów było odwiedzonych oraz jaki był maksymalny rozmiar kolejki:
- z punktu `(50, 42)` do `(100, 100)` w środowisku o rozmiarze 100x100. (Długość optymalnego rozwiązania to 110.)
- z punktu `(1, 1)` do `(20, 20)` w środowisku o rozmiarze 20x20.

Dla powyższego problemu 100x100: przeanalizuj wpływ na czas wykonania wynikły z użycia wolnych implementacji kolejek: ``SlowFIFOQueue``, ``SlowPriorityQueue``.
Jakie są zalety, a jakie wady algorytmu zachłannego?

Przydatne będzie magiczna operacja `%timeit`. Np.:
```python
%timeit BFS(problem)

X ms ± Y µs per loop (mean ± std. dev. of 7 runs, Z loops each)
```
które wskazuje przeciętny czas wykonania X ms, z odchyleniem standardowym Y mikrosekund przy wykonywaniu funkcji w grupach po Z razy.

Zdaj raport w odpowiednim polu.

In [ ]:
_, env = TB.get_instance(100)
first_problem = PathFindingProblem((50, 42), (100, 100), env.env.desc)

_, env = TB.get_instance(20)
second_problem = PathFindingProblem((1, 1), (20, 20), env.env.desc)

In [ ]:
%timeit BFS(first_problem)

1 loop, best of 5: 736 ms per loop


In [ ]:
# YOUR CODE HERE
def SlowBFS(problem):
  return search(problem, SlowFIFOQueue())

def SlowAstar(problem):
  return search(problem, SlowPriorityQueue(CostFunctionForAstar(problem)))

def SlowGreedy(problem):
  return search(problem, SlowPriorityQueue(CostFunctionForGreedy(problem)))

In [ ]:
print('BFS')
print('mapa 100x100')
%timeit BFS(first_problem)
print('odwiedzone stany: ' + str(len(visited)) + ', maksymalny rozmiar kolejki: ' + str(max_len))
print('mapa 20x20')
%timeit BFS(second_problem)
print('odwiedzone stany:  ' + str(len(visited)) + ', maksymalny rozmiar kolejki: ' + str(max_len))
print()
print('A*')
print('mapa 100x100')
%timeit astar(first_problem)
print('odwiedzone stany:  ' + str(len(visited)) + ', maksymalny rozmiar kolejki: ' + str(max_len))
print('mapa 20x20')
%timeit astar(second_problem)
print('odwiedzone stany:  ' + str(len(visited)) + ', maksymalny rozmiar kolejki: ' + str(max_len))
print()
print('Greedy')
print('mapa 100x100')
%timeit greedy(first_problem)
print('lodwiedzone stany:  ' + str(len(visited)) + ', maksymalny rozmiar kolejki: ' + str(max_len))
print('mapa 20x20')
%timeit greedy(second_problem)
print('odwiedzone stany:  ' + str(len(visited)) + ', maksymalny rozmiar kolejki: ' + str(max_len))
print()

print('Slow')
%timeit SlowBFS(first_problem)
%timeit SlowAstar(first_problem)
%timeit SlowGreedy(first_problem)

BFS
mapa 100x100
1 loop, best of 5: 752 ms per loop
odwiedzone stany: 4159, maksymalny rozmiar kolejki: 87
mapa 20x20
100 loops, best of 5: 2.2 ms per loop
odwiedzone stany:  165, maksymalny rozmiar kolejki: 9

A*
mapa 100x100
100 loops, best of 5: 10.9 ms per loop
odwiedzone stany:  431, maksymalny rozmiar kolejki: 58
mapa 20x20
1000 loops, best of 5: 1.08 ms per loop
odwiedzone stany:  90, maksymalny rozmiar kolejki: 18

Greedy
mapa 100x100
100 loops, best of 5: 4.57 ms per loop
lodwiedzone stany:  268, maksymalny rozmiar kolejki: 61
mapa 20x20
1000 loops, best of 5: 664 µs per loop
odwiedzone stany:  67, maksymalny rozmiar kolejki: 19

Slow
1 loop, best of 5: 776 ms per loop
100 loops, best of 5: 13.9 ms per loop
100 loops, best of 5: 5.13 ms per loop


Zdaj raport poniżej:

Algorytmy BFS, A* i zachłanny są kolejno coraz bardziej wydajne czasowo, oraz odwiedzają coraz mniej stanów. Maksymalny rozmiar kolejki wydaje się być najdłuższy w algorytmie BFS, natomiast przy algorytmach A* i zachłannego są one porównywalne. 

Jeżeli chodzi o czas wykonania implementacji wolnych kolejek, największe pogorszenie widzimy w algorytmie A*, natomiast dla dówch pozostałych różnice są nieznaczne. 

Wychodzi na to, że algorytmy zachłanne są bardzo skuteczne w problemach typu "znajdowanie najkrótszej ścieżki", ponieważ kierują się najbardziej korzystną opcją w danym kroku. Niestety takie sporzjenie na problem nie zawsze zwraca jednak optymalne rozwiązanie. Czasami trzeba uwzględnić informacje dotyczące ogólnego problemu, a nie tylko danego kroku. Natomiast w tym przypadku algorytm zachłanny działa najszybciej, daltego to właśnie jego użyję w zadaniu F.

# Zadanie F-inałowe: wygeneruj pełny plan
Wygeneruj pełny plan rozwiązania dla danego stanu w danym środowisku Taxi.

In [ ]:
def generate_plan(state, env):
    """Dostaje zakodowany stan i środowisko. Generuje najlepszą metodą i zwraca plan akcji agenta Taxi (0-5)."""
    # YOUR CODE HERE
    # dekodowanie
    row, col, client, destination = env.env.decode(state)
    locations = env.env.locs
    board = env.env.desc
    go_to_client = PathFindingProblem((row, col), locations[client], board)
    # w ramach sprawdzania
    print("stan początkowy: " + str((row, col)))
    print("klient: " + str(locations[client])) 
    print("cel: " + str(locations[destination]))
    # jedziemy po klienta 
    result = list()
    res1 = astar(go_to_client)
    result += get_solution(res1)
    result.append(4) # przyjechaliśmy po klienta, więc akcja 4 (Pickup)
    # następnie z klientem do celu
    go_to_goal = PathFindingProblem(res1.state, locations[destination], board)
    res2 = astar(go_to_goal)
    result += get_solution(res2)
    result.append(5) # 5 = Dropoff
    return result
                           

In [ ]:
state, env_6 = TB.get_instance(6)
print(env_6.render())
print("Przykładowy plan rozwiązania:")
print(generate_plan(state, env_6))

xxxxxxxx
xRxxx.Gx
x.....xx
x..xx.xx
x.....xx
x.xxxxxx
xY....Bx
xxxxxxxx

None
Przykładowy plan rozwiązania:
stan początkowy: (2, 1)
klient: (6, 1)
cel: (6, 6)
[0, 0, 0, 0, 4, 2, 2, 2, 2, 2, 5]


## Obserwacja
Czas na świętowanie! Teraz możesz obserwować, jak zachowuje się Twój agent.
Użyj ```state, env_6 = TB.get_instance(rozmiar)``` aby dostać losowe zadanie o zadanym `rozmiar`ze.
Wykonuj kolejne akcje agentem i wyrenderuj planszę po każdym ruchu.
Użyj ``TB.eval_plan(plan, env)`` by sprawdzić zysk uzyskany przez agenta rozwiązania.


## Testy generate_plan

In [ ]:
st_test, env_100 = TB.get_instance(100, 31)
res_test = TB.eval_plan(generate_plan(st_test, env_100), env_100)
print(res_test)
assert res_test == 688, "Poprawne rozwiązanie na tym seedzie ma zysk 688"
print("Sukces!")

stan początkowy: (50, 6)
klient: (100, 100)
cel: (1, 100)
688
Sukces!


In [ ]:
st_test, env_100 = TB.get_instance(100, 42)
res_test = TB.eval_plan(generate_plan(st_test, env_100), env_100)
print(res_test)
assert res_test == 694, "Poprawne rozwiązanie na tym seedzie ma zysk 688"
print("Sukces!")

stan początkowy: (39, 5)
klient: (100, 1)
cel: (1, 100)
694
Sukces!


# Bonus (nieobowiązkowy)
Zaimplementuj klasę TaxiProblem, która będzie przyjmowała stan obecny i stan docelowy prosto ze środowiska: (y, x, poz_klienta, poz_celu). Jej rozwiązaniem będzie całe rozwiązanie w środowisku. Pamiętaj, że gdy klient jest w taksówce, jego pozycja wynosi 4 (env.env.decode(...)[2] == 4).
Dobierz odpowiednio koszty i stwórz heurystykę, która będzie prowadziła do rozwiązania.
Ostateczny zysk nie powinien różnić się od poprzedniego rozwiązania.

In [ ]:
class TaxiProblem(Problem):
    """Klasa problemu taksówki."""

    def __init__(self, initial, env):
        """Parametry:
           - initial: zdekodowany stan (y, x, klient, cel),
           - env: środowisko Taxi-*."""
        # YOUR CODE HERE
        raise NotImplementedError()

    def actions(self, state):
        """Zwraca listę akcji ze stanu `state`.
           Powinny łatwo tłumaczyć się na akcje w środowisku taksówki."""
        
        # YOUR CODE HERE
        raise NotImplementedError()
        
    def result(self, state, action):
        "Stan, który jest rezultatem wykonania akcji `action` na stanie `state`."
        # YOUR CODE HERE
        raise NotImplementedError()

    def is_goal(self, state):
        "Zwraca `True`, gdy stan jest rozwiązaniem." 
        # YOUR CODE HERE
        raise NotImplementedError()

    def step_cost(self, state, action, result=None):
        "Koszt wykonania akcji `action` ze stanu `state`. `result` może być pomocniczne, gdy go znamy."
        # YOUR CODE HERE
        raise NotImplementedError()

In [ ]:
# Usuń rzucanie wyjątku, jeśli nie chcesz rozwiązywać.
# Heurystyki
# YOUR CODE HERE
raise NotImplementedError()

NotImplementedError: ignored

In [ ]:
#state, env = TB.get_instance(6, 42)
#problem = TaxiProblem(env.env.decode(state), env)
#plan = get_solution(???astar(problem))
#print(plan)
#TB.eval_plan(plan, env, verbosity=2)

In [ ]:
#state, env = TB.get_instance(100, 31)
#problem = TaxiProblem(env.env.decode(state), env)
#plan = get_solution(???astar(problem???))
#result = TB.eval_plan(plan, env, 0)
#print("Result == 688?", result == 688, result)

### Sprawdź, jak zachowuje się Twoja heurystyka
Ile mniej stanów odwiedza? Jak szybko działa? Czy jest poprawna (dopuszczalna i spójna)? Możesz to sprawdzić porównując rezultat na wielu przykładach.

# Na koniec
Pamiętaj, aby Twój notebook, którego prześlesz, wykonywał się w całości od początku do końca po kliknięciu "Run all". Zresetuj kernel, aby upewnić się, że tak właśnie jest.

Pobierz swój notebook w następujący sposób i wgraj go przez Moodle.

![](https://www.mimuw.edu.pl/~mm319369/priv/d73890416bec03ff3e2b3756af8c941c/images/download.png)